In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os


In [5]:
iris_data=pd.read_csv(r'/content/Iris.csv')

In [7]:
iris_data.head(7)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
5,6,5.4,3.9,1.7,0.4,Iris-setosa
6,7,4.6,3.4,1.4,0.3,Iris-setosa


In [10]:
iris_data.isnull().sum()

,0
Id,0
SepalLengthCm,0
SepalWidthCm,0
PetalLengthCm,0
PetalWidthCm,0
Species,0


In [11]:
iris_data.describe()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,75.500000,5.843333,3.054000,3.758667,1.198667
std,43.445368,0.828066,0.433594,1.764420,0.763161
min,1.000000,4.300000,2.000000,1.000000,0.100000
25%,38.250000,5.100000,2.800000,1.600000,0.300000
50%,75.500000,5.800000,3.000000,4.350000,1.300000
75%,112.750000,6.400000,3.300000,5.100000,1.800000
max,150.000000,7.900000,4.400000,6.900000,2.500000


In [16]:
iris_data.shape

(150, 6)

In [17]:
iris_data.drop("Id",axis=1,inplace=True)

In [18]:
iris_data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [20]:
iris_data['Species'].value_counts()


,count
Species,
Iris-setosa,50
Iris-versicolor,50
Iris-virginica,50


In [26]:
#split the iris_data to X_train,Y_train and X_test,Y_test using torch dataset
import torch
from torch import tensor
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision
from torchvision import transforms



In [27]:
#split iris_data to X_train,Y_train,X_test,Y_test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X=iris_data.drop("Species",axis=1).values

label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(iris_data['Species'])


X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42,stratify=Y)

X_train=torch.tensor(X_train,dtype=torch.float32)
X_test=torch.tensor(X_test,dtype=torch.float32)
Y_train=torch.tensor(Y_train,dtype=torch.long)
Y_test=torch.tensor(Y_test,dtype=torch.long)


print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test.shape)


X_train shape: torch.Size([120, 4])
Y_train shape: torch.Size([120])
X_test shape: torch.Size([30, 4])
Y_test shape: torch.Size([30])


In [28]:
class IrisDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]


train_dataset = IrisDataset(X_train, Y_train)
test_dataset = IrisDataset(X_test, Y_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [30]:
import torch.nn as nn
import torch.nn.functional as F

class IrisModel(nn.Module):
    def __init__(self):
        super(IrisModel, self).__init__()
        self.fc1 = nn.Linear(4, 10)
        self.fc2 = nn.Linear(10, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = IrisModel()

In [31]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [32]:
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for X_batch, Y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, Y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")


Epoch [1/100], Loss: 1.1083
Epoch [2/100], Loss: 0.9654
Epoch [3/100], Loss: 0.8530
Epoch [4/100], Loss: 0.7396
Epoch [5/100], Loss: 0.6691
Epoch [6/100], Loss: 0.5621
Epoch [7/100], Loss: 0.5003
Epoch [8/100], Loss: 0.4590
Epoch [9/100], Loss: 0.4244
Epoch [10/100], Loss: 0.3903
Epoch [11/100], Loss: 0.3654
Epoch [12/100], Loss: 0.3463
Epoch [13/100], Loss: 0.3182
Epoch [14/100], Loss: 0.2993
Epoch [15/100], Loss: 0.2771
Epoch [16/100], Loss: 0.2649
Epoch [17/100], Loss: 0.2445
Epoch [18/100], Loss: 0.2282
Epoch [19/100], Loss: 0.2133
Epoch [20/100], Loss: 0.2038
Epoch [21/100], Loss: 0.1902
Epoch [22/100], Loss: 0.1790
Epoch [23/100], Loss: 0.1778
Epoch [24/100], Loss: 0.1642
Epoch [25/100], Loss: 0.1652
Epoch [26/100], Loss: 0.1472
Epoch [27/100], Loss: 0.1484
Epoch [28/100], Loss: 0.1537
Epoch [29/100], Loss: 0.1652
Epoch [30/100], Loss: 0.1713
Epoch [31/100], Loss: 0.1362
Epoch [32/100], Loss: 0.1190
Epoch [33/100], Loss: 0.1312
Epoch [34/100], Loss: 0.1155
Epoch [35/100], Loss: 0

In [34]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for X_batch, Y_batch in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == Y_batch).sum().item()
        total += Y_batch.size(0)

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 93.33%


In [35]:
new_sample = torch.tensor([[5.1, 3.5, 1.4, 0.2]], dtype=torch.float32)
model.eval()
with torch.no_grad():
    output = model(new_sample)
    predicted_class = torch.argmax(output, dim=1).item()

print(f"Predicted class: {label_encoder.inverse_transform([predicted_class])[0]}")


Predicted class: Iris-setosa
